<a href="https://colab.research.google.com/github/har200509/NLP_TASK1/blob/main/Document_classifier_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/results.csv')

In [ ]:
df.head()

,document,text
0,E WAY BILL,e way bill e way bill e way bill date pm gen...
1,E WAY BILL,e way bill e way bill e way bill date pm gen...
2,E WAY BILL,blank e way bill e way bill e way bill date ...
3,E WAY BILL,e way bill e way bill e way bill date pm gen...
4,E WAY BILL,e way bill e way bill e way bill date pm gen...


In [ ]:
df['document'] = df['document'].str.lower()

In [ ]:
df['document'].value_counts()

,count
document,
delivery challan,230
e way bill,222
invoice,203
bill of entry,76
purchase order,70
debit note,60
credit note,57


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()

In [ ]:
import string
import re

In [ ]:
corpus_df=df['text'].tolist()

In [ ]:
len(corpus_df)

918

In [ ]:
for i in range(len(corpus_df)):
  for punc in string.punctuation:
    if punc in corpus_df[i]:
      corpus_df[i].replace(punc,"")

In [ ]:
pattern=r'^[a-z]{2,3}$'

In [ ]:
for i in range(len(corpus_df)):
    words = word_tokenize(corpus_df[i])
    words = [lemma.lemmatize(word.lower()) for word in words if word.lower() not in stopwords.words('english')]
    words = [word for word in words if not re.match(pattern, word)]
    corpus_df[i] = " ".join(words)

In [ ]:
corpus_df[0]

'e bill e bill e bill date generate l z j k industry valid valid part gstin supplier aaapw l z j k industry place dispatch mumbai maharashtra gstin recipient n steel limited village dolvi place delivery village dolvi maharashtra document document date transaction type regular value good code reason transportation outward supply transporter aacca c associated road carrier limited part b vehicle trans cewb multi info mode entered date enter road wadkhal aacca c'

In [ ]:
document_df=[]
for para in corpus_df:
  words=para.split(" ")
  document_df.append(words)

In [ ]:
document_df[0]

['e',
 'bill',
 'e',
 'bill',
 'e',
 'bill',
 'date',
 'generate',
 'l',
 'z',
 'j',
 'k',
 'industry',
 'valid',
 'valid',
 'part',
 'gstin',
 'supplier',
 'aaapw',
 'l',
 'z',
 'j',
 'k',
 'industry',
 'place',
 'dispatch',
 'mumbai',
 'maharashtra',
 'gstin',
 'recipient',
 'n',
 'steel',
 'limited',
 'village',
 'dolvi',
 'place',
 'delivery',
 'village',
 'dolvi',
 'maharashtra',
 'document',
 'document',
 'date',
 'transaction',
 'type',
 'regular',
 'value',
 'good',
 'code',
 'reason',
 'transportation',
 'outward',
 'supply',
 'transporter',
 'aacca',
 'c',
 'associated',
 'road',
 'carrier',
 'limited',
 'part',
 'b',
 'vehicle',
 'trans',
 'cewb',
 'multi',
 'info',
 'mode',
 'entered',
 'date',
 'enter',
 'road',
 'wadkhal',
 'aacca',
 'c']

In [ ]:
len(document_df)

918

In [ ]:
import pickle

In [ ]:
%pip install gensim

In [ ]:
import gensim

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
class Word2vec(object):
    def __init__(self, documents):
        self.documents = documents

    def train_w2v(self):
        w2v_model = Word2Vec(self.documents, vector_size=200, window=10, min_count=1, alpha=0.03, min_alpha=0.0007, sg=1)

        w2v_model.build_vocab(self.documents, progress_per=10000, update=True)
        w2v_model.train(self.documents, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)

        with open('w2v_model.bin', 'wb') as b:
            pickle.dump(w2v_model, b, protocol=pickle.HIGHEST_PROTOCOL)

        return w2v_model

In [ ]:
import numpy as np

def get_avg_w2v_vectors(documents, model):
    vector_size = model.vector_size
    avg_vectors = []
    for doc in documents:
        valid_words = [word for word in doc if word in model.wv.key_to_index]
        if valid_words:
            vecs = [model.wv[word] for word in valid_words]
            avg_vec = np.mean(vecs, axis=0)
        else:
            avg_vec = np.zeros(vector_size)
        avg_vectors.append(avg_vec)

    return np.array(avg_vectors)

In [ ]:
model=Word2vec(document_df)
model.train_w2v()

In [ ]:
w2v_mod=gensim.models.Word2Vec.load('w2v_model.bin')

In [ ]:
len(w2v_mod.wv.key_to_index)

9122

In [ ]:
avg_w2v_df=get_avg_w2v_vectors(document_df,w2v_mod)

In [ ]:
len(avg_w2v_df)

918

In [ ]:
df['document'].value_counts()

,count
document,
delivery challan,230
e way bill,222
invoice,203
bill of entry,76
purchase order,70
debit note,60
credit note,57


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
df['document']=le.fit_transform(df['document'])

In [ ]:
df['document'].value_counts()

,count
document,
3,230
4,222
5,203
0,76
6,70
2,60
1,57


In [ ]:
doc_label={0:'bill of entry',1:'credit note',2:'debit note',3:'delivery challan',4:'e way bill',5:'invoice',6:'purchase order	'}

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(avg_w2v_df,df['document'],test_size=0.2,random_state=42,stratify=df['document'])

In [ ]:
from sklearn.svm import SVC

svc_model=SVC(kernel='linear')

svc_model.fit(x_train,y_train)

SVC(kernel='linear')

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(y_test,svc_model.predict(x_test))

0.9782608695652174

In [ ]:
confusion_matrix(y_test,svc_model.predict(x_test))

array([[15,  0,  0,  0,  0,  0,  0],
       [ 0,  9,  1,  0,  0,  1,  0],
       [ 0,  0, 12,  0,  0,  0,  0],
       [ 0,  0,  0, 46,  0,  0,  0],
       [ 0,  0,  0,  0, 45,  0,  0],
       [ 0,  2,  0,  0,  0, 39,  0],
       [ 0,  0,  0,  0,  0,  0, 14]])

In [ ]:
demo2_df=pd.read_csv('/content/Demo_2.csv')

In [ ]:
demo2_df.head()

,document,text
0,credit note,manickbag automobiles p ltd tata motors vid...
1,e way bill,e way bill e way bill details eway bill gene...
2,credit note,credit note e invoice irn baee f acb b de b e...
3,credit note,credit note airtel original recipient mailing ...
4,credit note,credit note e invoice irn ee cea ebadd c c b ...


In [ ]:
demo2_df.columns=['document','text']

In [ ]:
corpus_demo2=demo2_df['text'].tolist()

In [ ]:
len(corpus_demo2)

176

In [ ]:
for i in range(len(corpus_demo2)):
  for punc in string.punctuation:
    if punc in corpus_demo2[i]:
      corpus_demo2[i].replace(punc,"")

In [ ]:
pattern=r'^[a-z]{2,3}$'

In [ ]:
for i in range(len(corpus_demo2)):
    words = word_tokenize(corpus_demo2[i])
    words = [lemma.lemmatize(word.lower()) for word in words if word.lower() not in stopwords.words('english')]
    words = [word for word in words if not re.match(pattern, word)]
    corpus_demo2[i] = " ".join(words)

In [ ]:
corpus_demo2[0]

'manickbag automobile p tata motor vidyanagar hubli phone passenger dealer help line gstin aadcm k u name manickbag automobile private limited dharwad road belgaum karnataka india invoice type manickbag automobile p hubballi s stamp credit note referance number manick bill tata capital financial service limited invoice date imaphb tcfsl date tata altroz location hubli npgr ship maxim quadras thomas quadras shakhambri apartment narayanapur g mahalaxmi temple dharwad hubli dharwad hubli dharwad dharwad department sale karnataka supportin description rate amount g bill invoice amount registration amount insurance aadcm k service regn aadcm aadcm k service category rupee word fourteen thousand five hundred fortytwo rupee total manickbag note logistics charge manick hubling authorised signatory'

In [ ]:
document_demo2=[]
for para in corpus_demo2:
  words=para.split(" ")
  document_demo2.append(words)

In [ ]:
document_demo2[0]

['manickbag',
 'automobile',
 'p',
 'tata',
 'motor',
 'vidyanagar',
 'hubli',
 'phone',
 'passenger',
 'dealer',
 'help',
 'line',
 'gstin',
 'aadcm',
 'k',
 'u',
 'name',
 'manickbag',
 'automobile',
 'private',
 'limited',
 'dharwad',
 'road',
 'belgaum',
 'karnataka',
 'india',
 'invoice',
 'type',
 'manickbag',
 'automobile',
 'p',
 'hubballi',
 's',
 'stamp',
 'credit',
 'note',
 'referance',
 'number',
 'manick',
 'bill',
 'tata',
 'capital',
 'financial',
 'service',
 'limited',
 'invoice',
 'date',
 'imaphb',
 'tcfsl',
 'date',
 'tata',
 'altroz',
 'location',
 'hubli',
 'npgr',
 'ship',
 'maxim',
 'quadras',
 'thomas',
 'quadras',
 'shakhambri',
 'apartment',
 'narayanapur',
 'g',
 'mahalaxmi',
 'temple',
 'dharwad',
 'hubli',
 'dharwad',
 'hubli',
 'dharwad',
 'dharwad',
 'department',
 'sale',
 'karnataka',
 'supportin',
 'description',
 'rate',
 'amount',
 'g',
 'bill',
 'invoice',
 'amount',
 'registration',
 'amount',
 'insurance',
 'aadcm',
 'k',
 'service',
 'regn',
 '

In [ ]:
model_demo2=Word2vec(document_demo2)
model_demo2.train_w2v()

KeyboardInterrupt: 

In [ ]:
w2v_mod_demo2=gensim.models.Word2Vec.load('w2v_model.bin')

In [ ]:
len(w2v_mod.wv.key_to_index)

9122

In [ ]:
len(w2v_mod_demo2.wv.key_to_index)

9122

In [ ]:
avg_w2v_df_demo=get_avg_w2v_vectors(document_demo2,w2v_mod)

In [ ]:
len(avg_w2v_df_demo)

176

In [ ]:
predicted=svc_model.predict(avg_w2v_df_demo)

In [ ]:
len(predicted)

176

In [ ]:
doc_label={0:'bill of entry',1:'credit note',2:'debit note',3:'delivery challan',4:'e way bill',5:'invoice',6:'purchase order	'}

In [ ]:
count=0;
for i in range(len(predicted)):
  value=doc_label.get(predicted[i])
  if value!=demo2_df.iloc[i,0]:
    print(f"{demo2_df.iloc[i, 0]} : {value}")
    count+=1
print(count)

credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit note : invoice
credit not

In [ ]:
demo2_df['document'].value_counts()

,count
document,
credit note,71
e way bill,64
delivery challan,28
purchase order,8
debit note,5
